In [3]:
import sys
sys.path.append('..')

import torch
import os
import scipy.io
import pandas as pd
import numpy as np

from PIL import Image
from tqdm import tqdm
from torchvision import transforms, datasets
from torchvision import models as torchvision_models

import utils

In [11]:
arch = 'resnet50'
dat = 'obj'
ckpt_pth = f'/om2/user/amarvi/dino/saved_models/{dat}400_dino-r50/checkpoint.pth'

model = torchvision_models.__dict__[arch]()
model.cuda()
model.eval()
utils.load_pretrained_weights(model, ckpt_pth, 'student', arch, 0)

print(f"Model {arch} built.")

Take key student in provided checkpoint dict
Pretrained weights found at /om2/user/amarvi/dino/saved_models/obj400_dino-r50/checkpoint.pth and loaded with msg: _IncompatibleKeys(missing_keys=['fc.weight', 'fc.bias'], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])
Model resnet50 built.


In [6]:
transform = transforms.Compose([
    transforms.Resize(256, interpolation=3),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
]) 

In [12]:
img_folder = f'/om2/user/amarvi/FACE/data/behav_{dat}'
img_list = []

for jpg_name in tqdm(os.listdir(img_folder)):
    img_pth = os.path.join(img_folder, jpg_name)
    img = Image.open(img_pth).convert('RGB')
    inpt_img = transform(img)
    img_list.append(inpt_img)
    
batch_tensor = torch.stack(img_list).cuda()    

100%|██████████| 200/200 [00:05<00:00, 36.18it/s]


In [13]:
activations = {}
hooks = []
register_hooks(model)

with torch.no_grad():
    _ = model(batch_tensor)

In [14]:
# cols = ['model', 'dataset', 'size', 'image_idx', 'activation']
# df = pd.DataFrame(columns=cols)
for idx, (layer, activation) in enumerate(activations.items()):
    df.loc[len(df)] = {'model': 'dino', 'dataset': dat, 'size': '400', 'image_idx': layers[idx], 'activation': activation.detach().cpu().numpy()}

In [16]:
df.to_pickle('/om2/user/amarvi/dino/saved_models/mts_dino-r50_activations.pkl')

In [7]:
def hook_fn(module, input, output):
    activations[module] = output

def register_hooks(model):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.ReLU):
            hook = module.register_forward_hook(hook_fn)
            hooks.append(hook)
            
layers = []
for name, module in model.named_modules():
    if isinstance(module, torch.nn.ReLU):
        layers.append(name)

In [17]:
f = scipy.io.loadmat('/om2/user/amarvi/FACE/data/data_up.mat')

triplet = f['data_up'][0][0][0]
perf = f['data_up'][0][0][1]

print(triplet.shape, perf.shape)

(3, 1560) (1, 1560)


In [21]:
column_names = ['model', 'size', 'dataset', 'layer', 'results', 'accuracy', 'ci']
res_df = pd.DataFrame(columns=column_names)

for index, row in tqdm(df.iterrows()):
    act = row['activation']
    dat = row['dataset']
    count = 0
    bstrap = []

    for idx, trio in enumerate(triplet.transpose()):
        trio = trio-1
        gt = trio//5
        [targ, m1, m2] = trio

        dist1 = np.linalg.norm(act[targ] - act[m1])
        dist2 = np.linalg.norm(act[targ] - act[m2])
        model_choice = np.argmax(np.array([dist1, dist2])) + 1
        correct_choice = np.where(gt != gt[0])[0]


        bstrap.append(int(model_choice == correct_choice))
        if model_choice != correct_choice:
            count += 1
            
    bstrap = np.array(bstrap)
    n_bootstrap = 10000
    bootstrap_samples = np.random.choice(bstrap, size=(len(bstrap), n_bootstrap), replace=True)
    bootstrap_sample_means = np.mean(bootstrap_samples, axis=0)

    ci_lower = np.percentile(bootstrap_sample_means, 2.5)
    ci_upper = np.percentile(bootstrap_sample_means, 97.5)


    acc = 1 - count/len(triplet[0])
#     print("CI:", (ci_lower, ci_upper), 'Mean:', np.mean(bstrap))
    print(dat, row['image_idx'], acc)

    res_df.loc[len(res_df)] = {'model': 'dino-r50', 'size': '400', 'dataset': dat, 'layer': row['image_idx'], 'results': bstrap, 'accuracy':np.mean(bstrap), 'ci': (ci_lower, ci_upper)}

1it [00:02,  2.31s/it]

face relu 0.5301282051282051


2it [00:04,  2.31s/it]

face layer1.0.relu 0.5775641025641025


3it [00:06,  2.31s/it]

...

obj layer4.0.relu 0.7884615384615384


33it [00:41,  1.34it/s]

obj layer4.1.relu 0.8051282051282052


34it [00:42,  1.25s/it]

obj layer4.2.relu 0.8038461538461539


In [22]:
res_df.to_pickle('/om2/user/amarvi/dino/saved_models/mts_dino-r50_results.pkl')